In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import StackingClassifier

In [2]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [3]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [4]:
#import models with pickle
import pickle
clf1 = pickle.load(open('models/catboost_model.sav', 'rb'))
clf2 = pickle.load(open('models/extra_model.sav', 'rb'))
clf3 = pickle.load(open('models/svm_model.sav', 'rb'))
clf4 = pickle.load(open('models/xgboost_model.sav', 'rb'))
clf5 = pickle.load(open('models/random_forest_model.sav', 'rb'))

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
final_estimator = SVC(probability = True)
eclf2 = StackingClassifier(estimators=[ ('catboost',clf1), ('extra', clf2), ('svm', clf3), ('xgboost', clf4), ('rf',clf5)], final_estimator=final_estimator)
scores = cross_val_score(eclf2, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Learning rate set to 0.235163
Learning rate set to 0.235166
0:	learn: 0.6743220	total: 2.19s	remaining: 10m 54s
Learning rate set to 0.235166
1:	learn: 0.6870266	total: 3.86s	remaining: 9m 35s
0:	learn: 0.6675623	total: 4.24s	remaining: 21m 7s
2:	learn: 0.7109699	total: 4.92s	remaining: 8m 6s
Learning rate set to 0.235163
3:	learn: 0.7212314	total: 6.47s	remaining: 7m 59s
1:	learn: 0.6775769	total: 6.11s	remaining: 15m 10s
4:	learn: 0.7295382	total: 8.21s	remaining: 8m 4s
2:	learn: 0.7039570	total: 7.68s	remaining: 12m 40s
3:	learn: 0.7173913	total: 8.82s	remaining: 10m 52s
Learning rate set to 0.235166
0:	learn: 0.6893014	total: 6.36s	remaining: 31m 41s
5:	learn: 0.7397997	total: 9.99s	remaining: 8m 9s
4:	learn: 0.7291158	total: 10.5s	remaining: 10m 20s
1:	learn: 0.7056668	total: 7.8s	remaining: 19m 22s
6:	learn: 0.7405326	total: 11.4s	remaining: 7m 57s
5:	learn: 0.7369321	total: 11.9s	remaining: 9m 42s
2:	learn: 0.7059111	total: 9.19s	remaining: 15m 9s
7:	learn: 0.7463963	total: 12.7

[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed: 152.1min remaining: 228.2min


[CV] END ................................................... total time=152.3min
[CV] END ................................................... total time=152.5min
[CV] END ................................................... total time=152.5min


[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed: 152.6min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed: 152.6min finished


In [6]:
print(scores.mean())
print(scores.std())
#0.7148702880620725 / 0.01066740914333765
#0.7287474034701859 / 0.008069266211889234
#0.7346098332111438 / 0.011019933020849262 578 features with logistic regression
#0.7379316730816228 / 0.012364254009713383 578 features with svm
#0.816495028409091 / 0.0054742345511349854 1000 features

0.816495028409091
0.0054742345511349854


In [7]:
eclf2.fit(x_train, np.ravel(y_train))
y_test = eclf2.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_ensemble_stacking.csv', index=False)

Learning rate set to 0.237857
0:	learn: 0.6912253	total: 965ms	remaining: 4m 48s
1:	learn: 0.6922025	total: 1.94s	remaining: 4m 49s
2:	learn: 0.7125269	total: 2.93s	remaining: 4m 50s
3:	learn: 0.7293336	total: 3.71s	remaining: 4m 34s
4:	learn: 0.7346101	total: 4.7s	remaining: 4m 37s
5:	learn: 0.7404729	total: 5.61s	remaining: 4m 34s
6:	learn: 0.7475083	total: 6.44s	remaining: 4m 29s
7:	learn: 0.7547391	total: 7.21s	remaining: 4m 23s
8:	learn: 0.7600156	total: 8.01s	remaining: 4m 19s
9:	learn: 0.7645105	total: 8.75s	remaining: 4m 13s
10:	learn: 0.7672464	total: 9.51s	remaining: 4m 9s
11:	learn: 0.7647059	total: 10.3s	remaining: 4m 6s
12:	learn: 0.7723275	total: 11s	remaining: 4m 2s
13:	learn: 0.7735001	total: 11.8s	remaining: 4m 1s
14:	learn: 0.7777995	total: 12.6s	remaining: 3m 59s
15:	learn: 0.7848349	total: 13.4s	remaining: 3m 57s
16:	learn: 0.7865937	total: 14.1s	remaining: 3m 55s
17:	learn: 0.7912840	total: 14.9s	remaining: 3m 52s
18:	learn: 0.7934337	total: 15.7s	remaining: 3m 51s